<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/05_Transformers_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de Texto usando Transformers

Este notebook demuestra cómo usar modelos Transformer para generar texto en español.

Utilizaremos el modelo GPT2 específicamente ajustado para español.

##1) Importar librerías necesarias

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

##2) Cargar el modelo y tokenizador
Usamos 'datificate/gpt2-small-spanish', un modelo más ligero entrenado para español

In [2]:
model_name = "datificate/gpt2-small-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

##3) Función para generar texto


In [3]:
def generar_texto(prompt, max_length=100, num_return_sequences=1):
    # Tokenizar el texto de entrada
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generar texto
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decodificar y mostrar los resultados
    for i, output in enumerate(outputs):
        texto_generado = tokenizer.decode(output, skip_special_tokens=True)
        print(f"\nTexto generado {i+1}:")
        print(texto_generado)
        print("-" * 50)

##4) Probar diferentes tipos de generación

In [4]:
# Ejemplo 1: Completar una historia
print("=== Completando una historia ===")
prompt_historia = "Había una vez un científico que descubrió"
generar_texto(prompt_historia, max_length=150)

# Ejemplo 2: Generar una receta
print("\n=== Generando una receta ===")
prompt_receta = "Para preparar una tortilla española necesitas"
generar_texto(prompt_receta, max_length=200)

# Ejemplo 3: Múltiples variaciones
print("\n=== Generando múltiples variaciones ===")
prompt_variaciones = "El futuro de la inteligencia artificial"
generar_texto(prompt_variaciones, max_length=100, num_return_sequences=3)

=== Completando una historia ===

Texto generado 1:
Había una vez un científico que descubrió la verdad sobre el origen de la materia y la creación de un cuerpo humano. Sin embargo, debido a los acontecimientos que tuvo que suceder durante la guerra, el Imperio de los Estados Unidos perdió su independencia.

Al final del juego, la serie vuelve a la normalidad. La historia continúa hasta la primera temporada. El jugador toma el papel de James Ray, un agente de inteligencia que trabaja para la CIA, y es el único miembro del equipo que ha tenido éxito en descubrir la verdadera identidad de las personas que han sido asesinadas por el gobierno de Estados de Unidos. Ray es un hombre de negocios de clase alta y posee un sentido de pertenencia. Se dice que Ray tiene un corazón de oro, lo que le permite
--------------------------------------------------

=== Generando una receta ===

Texto generado 1:
Para preparar una tortilla española necesitas una cierta cantidad de grasa y una cantidad sign

##5) Funcion Interactiva

In [5]:
def generar_texto_interactivo():
    prompt = input("\nEscribe un comienzo para generar texto: ")
    longitud = int(input("¿Cuántos caracteres quieres generar? (recomendado: 100-200): "))
    variaciones = int(input("¿Cuántas variaciones quieres ver? (1-3): "))

    print("\nGenerando texto...")
    generar_texto(prompt, max_length=longitud, num_return_sequences=variaciones)

# Probar la función interactiva
print("\n=== Prueba la generación de texto ===")
generar_texto_interactivo()


=== Prueba la generación de texto ===

Escribe un comienzo para generar texto: Hola buenos días, 
¿Cuántos caracteres quieres generar? (recomendado: 100-200): 100
¿Cuántas variaciones quieres ver? (1-3): 1

Generando texto...

Texto generado 1:
Hola buenos días, 
"Lola bella días"

La canción "Lolli" (del álbum "Los últimos días de mi vida") es una canción de la banda estadounidense de "hard rock" estadounidense Led Zeppelin, publicada por primera vez en 1963. Esta canción se convirtió en un éxito de ventas en los Estados Unidos y en el Reino Unido.
Este sencillo es un clásico de Led en la década de 1960, al igual que otros grandes éxitos de The Beatles. Fue
--------------------------------------------------
